In [1]:
# Import required libraries
import os
import pickle
import pandas as pd
import numpy as np
from musixmatch import Musixmatch
from tqdm import tqdm_notebook # For creating progress bars
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
# https://jovian.ai/zhangxm963/spotify-dataframe/v/39/files?filename=spotify-dataset-19212020-160k-tracks/data.csv
# Read the dataset
sdf = pd.read_csv("data/spotify_data.csv")
sdf.shape

(174389, 19)

In [11]:
# Filter only songs between 2001 and 2019(inclusive); as 2020 and 2021 songs will be used for analyzing at front end
spotify_df1 = sdf[(sdf.year >= 2001) & (sdf.year < 2020)]
spotify_df1.drop_duplicates('id', keep='first')

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
9071,0.39400,['Gerry & The Pacemakers'],0.484,160187,0.265,0,6catF1lDhNTjjGa2GxRQNN,0.000000,0,0.1490,-11.101,1,You'll Never Walk Alone - Mono; 2002 Remaster,55,2008-02-11,0.0322,113.564,0.285,2008
9081,0.25500,['Gerry & The Pacemakers'],0.405,141987,0.365,0,4aSw1QJIMwYSoDEgzgdCJL,0.000005,6,0.1630,-10.226,0,Ferry Cross the Mersey - Mono; 2002 Remaster,39,2008-02-11,0.0289,104.536,0.588,2008
9085,0.40600,['Gerry & The Pacemakers'],0.477,157093,0.352,0,0ZMMtH875IR2TfkyC4PolD,0.000000,1,0.1220,-14.165,1,Don't Let the Sun Catch You Crying (Main) - Mono,34,2008-02-11,0.0300,106.773,0.478,2008
9087,0.88700,['Frank Sinatra'],0.319,187333,0.201,0,1hx7X9cMXHWJjknb9O6Ava,0.000000,7,0.9040,-17.796,1,The September Of My Years - Live At The Sands ...,27,2018-05-04,0.0623,117.153,0.239,2018
9091,0.93800,['Frank Sinatra'],0.269,236800,0.129,0,19oquvXf3bc65GSqtPYA5S,0.000005,7,0.6830,-18.168,0,It Was A Very Good Year - Live At The Sands Ho...,26,2018-05-04,0.0576,82.332,0.160,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174116,0.01380,['Workout Music '],0.734,277493,0.809,0,4ZzfKwqPfy02fpQxuh9Zz3,0.012600,0,0.0635,-5.655,0,RITMO (Bay Boys For Life) [Remix],14,2019-12-26,0.1150,128.015,0.965,2019
174118,0.06540,['Workout Music '],0.533,287987,0.879,0,3O1lskEk6MF8bEQ76Q7Cmv,0.000000,5,0.0147,-3.736,1,Beautiful People (Remix),14,2019-12-26,0.0492,129.943,0.398,2019
174120,0.00358,['Workout Music '],0.764,268120,0.828,0,4fBQiIB6X7sE5RWK7mZq8N,0.005500,2,0.0763,-3.625,1,Tusa (Remix),16,2019-12-26,0.0810,127.994,0.931,2019
174124,0.98100,['October'],0.520,226603,0.198,0,1g6Zm6BiN3WktHGzu0ZKAh,0.909000,1,0.1020,-23.823,1,A Good Year - 2020 Remastered,5,2013,0.0483,123.976,0.272,2013


In [12]:
spotify_df2 = sdf[sdf.year >= 2020]
spotify_df2.drop_duplicates('id', keep='first')

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
9111,0.881,['Joni Mitchell'],0.644,313093,0.212,0,55qyghODi24yaDgKBI6lx0,0.000022,11,0.7980,-14.118,1,"The Circle Game - Live at The 2nd Fret, Philad...",19,2020-10-30,0.0347,117.072,0.441,2020
9117,0.955,['Joni Mitchell'],0.627,295093,0.184,0,00xemFYjQNRpOlPhVaLAHa,0.000162,1,0.0986,-15.533,1,"Urge For Going - Live at The 2nd Fret, Philade...",19,2020-10-30,0.0450,115.864,0.299,2020
9119,0.888,['Joni Mitchell'],0.581,183440,0.331,0,2lm5FQJRHvc3rUN5YHpEWj,0.000015,6,0.1470,-14.087,1,What's The Story Mr. Blue - Live at The 2nd Fr...,19,2020-10-30,0.2430,88.303,0.642,2020
9121,0.930,['Joni Mitchell'],0.442,147907,0.399,0,26g4FBGTB9YEj7q4HlblFf,0.000499,6,0.9120,-12.661,1,"Brandy Eyes - Live at The 2nd Fret, Philadelph...",19,2020-10-30,0.0780,121.662,0.554,2020
9129,0.949,['Joni Mitchell'],0.570,64173,0.176,0,05sxkljafFBW2vEnVczQy1,0.000000,6,0.1470,-22.676,0,Intro To Urge For Going - Live at The 2nd Fret...,19,2020-10-30,0.2990,135.687,0.348,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174379,0.795,['Alessia Cara'],0.429,144720,0.211,0,45XnLMuqf3vRfskEAMUeCH,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.228,2021
174381,0.795,['Alessia Cara'],0.429,144720,0.211,0,4pPFI9jsguIh3wC7Otoyy8,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.228,2021
174383,0.795,['Alessia Cara'],0.429,144720,0.211,0,52YtxLVUyvtiGPxwwxayHZ,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.228,2021
174385,0.795,['Alessia Cara'],0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.228,2021


In [13]:
# Randomly select 12K rows
spotify_df1 = spotify_df1.sample(12000, random_state=7).reset_index()
spotify_df1

,index,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,75771,0.057400,['Dmitriy Rublev'],0.612,423882,0.835,0,4pKfJjAisHCmqDLCLGwBKq,0.954000,1,0.2030,-10.357,0,New Year 2016,0,2017-02-08,0.0338,128.010,0.257,2017
1,55117,0.132000,"['Madvillain', 'Madlib', 'MF DOOM']",0.273,90133,0.670,1,6xSBcheWUUOJPAf0mQTQ80,0.001910,7,0.6660,-10.202,1,Operation Lifesaver aka Mint Test,57,2004-03-24,0.4210,183.170,0.721,2004
2,74567,0.587000,['Hot Tuna'],0.682,510320,0.506,0,5tHGkByx23eb8apgtAVUv4,0.674000,4,0.7120,-15.009,1,99 Year Blues,0,2011-03-15,0.0293,99.808,0.794,2011
3,92097,0.018400,['Jasper Arnou'],0.860,421935,0.300,0,7oeffmNa73ruTvx86tw3aq,0.426000,9,0.0907,-17.490,0,Bumbershoot - Saint Row Remix,0,2012-01-26,0.1110,124.007,0.573,2012
4,170476,0.164000,['Enon'],0.586,169533,0.862,0,2Q6QyI9G48Et2mIBs13gTW,0.000126,5,0.1290,-5.041,1,Natural Disasters,37,2002-06-04,0.0466,147.704,0.736,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,76000,0.096700,['Playboi Carti'],0.755,193973,0.552,1,5nayhWICkQGMTkisxVMbRw,0.000000,9,0.1580,-5.604,1,FlatBed Freestyle,70,2018-05-11,0.2880,153.157,0.402,2018
11996,170955,0.000473,['Stefan Hellstrom'],0.871,460684,0.562,0,0zueYVrNbxyrdZIje7PxQe,0.887000,8,0.1060,-7.956,0,Blue Carbon - Kriece Remix,0,2014-10-10,0.0979,126.001,0.244,2014
11997,92809,0.000340,"['Mark Sixma', 'Emma Hewitt']",0.524,95453,0.969,0,46LxyJXPgM21srcWWbBW0g,0.659000,10,0.0817,-5.222,0,Restless Hearts (Mix Cut) - Club Mix,4,2015-12-18,0.0449,132.044,0.607,2015
11998,76302,0.159000,['Strawberry Guy'],0.477,208500,0.629,0,45BBlVHECwB0uNt7BsJ97r,0.440000,5,0.3740,-7.408,0,Mrs Magic,70,2019-07-22,0.0315,132.535,0.357,2019


In [14]:
spotify_df = pd.concat([spotify_df1, spotify_df2], ignore_index=True)
spotify_df.drop(columns=["index"], inplace=True)

In [15]:
spotify_df["year"].value_counts()

2020    4294
2021    1840
2018     928
2013     896
2019     827
2015     783
2016     777
2014     736
2017     728
2010     706
2012     681
2002     671
2011     616
2008     590
2009     566
2006     472
2003     436
2007     433
2005     428
2004     375
2001     351
Name: year, dtype: int64

In [16]:
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18134 entries, 0 to 18133
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      18134 non-null  float64
 1   artists           18134 non-null  object 
 2   danceability      18134 non-null  float64
 3   duration_ms       18134 non-null  int64  
 4   energy            18134 non-null  float64
 5   explicit          18134 non-null  int64  
 6   id                18134 non-null  object 
 7   instrumentalness  18134 non-null  float64
 8   key               18134 non-null  int64  
 9   liveness          18134 non-null  float64
 10  loudness          18134 non-null  float64
 11  mode              18134 non-null  int64  
 12  name              18134 non-null  object 
 13  popularity        18134 non-null  int64  
 14  release_date      18134 non-null  object 
 15  speechiness       18134 non-null  float64
 16  tempo             18134 non-null  float6

In [18]:
# Drop rows with none value
spotify_df = spotify_df.dropna()

In [19]:
def get_artist_name(text):
    text = text.split(",")[0]
    return text

def remove_extra_chars(text):
    text = text.replace("'", "").replace("[", "").replace("]", "")
    return text
    
spotify_df["artist_name"] = spotify_df["artists"].apply(get_artist_name)
spotify_df["artist_name"] = spotify_df["artist_name"].apply(remove_extra_chars)
spotify_df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,artist_name
0,0.05740,['Dmitriy Rublev'],0.612,423882,0.835,0,4pKfJjAisHCmqDLCLGwBKq,0.954000,1,0.2030,-10.357,0,New Year 2016,0,2017-02-08,0.0338,128.010,0.257,2017,Dmitriy Rublev
1,0.13200,"['Madvillain', 'Madlib', 'MF DOOM']",0.273,90133,0.670,1,6xSBcheWUUOJPAf0mQTQ80,0.001910,7,0.6660,-10.202,1,Operation Lifesaver aka Mint Test,57,2004-03-24,0.4210,183.170,0.721,2004,Madvillain
2,0.58700,['Hot Tuna'],0.682,510320,0.506,0,5tHGkByx23eb8apgtAVUv4,0.674000,4,0.7120,-15.009,1,99 Year Blues,0,2011-03-15,0.0293,99.808,0.794,2011,Hot Tuna
3,0.01840,['Jasper Arnou'],0.860,421935,0.300,0,7oeffmNa73ruTvx86tw3aq,0.426000,9,0.0907,-17.490,0,Bumbershoot - Saint Row Remix,0,2012-01-26,0.1110,124.007,0.573,2012,Jasper Arnou
4,0.16400,['Enon'],0.586,169533,0.862,0,2Q6QyI9G48Et2mIBs13gTW,0.000126,5,0.1290,-5.041,1,Natural Disasters,37,2002-06-04,0.0466,147.704,0.736,2002,Enon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0.00917,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.186,2020,DJ Combo
18130,0.79500,['Alessia Cara'],0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.228,2021,Alessia Cara
18131,0.80600,['Roger Fly'],0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.714,2020,Roger Fly
18132,0.92000,['Taylor Swift'],0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.320,2021,Taylor Swift


In [20]:
spotify_df = spotify_df.rename(columns={"release_date": "rel_date", "name": "track_name"})
spotify_df = spotify_df.drop(columns=["artists"])

In [21]:
spotify_df = spotify_df[["id", "artist_name", "key", "mode", "track_name", "rel_date", "popularity", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"]]
spotify_df.head()

,id,artist_name,key,mode,track_name,rel_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,4pKfJjAisHCmqDLCLGwBKq,Dmitriy Rublev,1,0,New Year 2016,2017-02-08,0,0.0574,0.612,423882,0.835,0.954000,0.2030,-10.357,0.0338,128.010,0.257
1,6xSBcheWUUOJPAf0mQTQ80,Madvillain,7,1,Operation Lifesaver aka Mint Test,2004-03-24,57,0.1320,0.273,90133,0.670,0.001910,0.6660,-10.202,0.4210,183.170,0.721
2,5tHGkByx23eb8apgtAVUv4,Hot Tuna,4,1,99 Year Blues,2011-03-15,0,0.5870,0.682,510320,0.506,0.674000,0.7120,-15.009,0.0293,99.808,0.794
3,7oeffmNa73ruTvx86tw3aq,Jasper Arnou,9,0,Bumbershoot - Saint Row Remix,2012-01-26,0,0.0184,0.860,421935,0.300,0.426000,0.0907,-17.490,0.1110,124.007,0.573
4,2Q6QyI9G48Et2mIBs13gTW,Enon,5,1,Natural Disasters,2002-06-04,37,0.1640,0.586,169533,0.862,0.000126,0.1290,-5.041,0.0466,147.704,0.736


In [22]:
billboard_df = pd.read_csv("data/billboard_scraped.csv")
billboard_df.head()

,Unnamed: 0,id,artist_name,key,mode,track_name,rel_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,2000,68,0.00118,0.541,216067.0,0.864,0.00000,0.0896,-4.915,0.0357,124.557,0.435
1,1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',2001-06-05,70,0.26300,0.652,210200.0,0.609,0.00101,0.2330,-7.519,0.0370,95.986,0.482
2,2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,2001-01-01,67,0.01740,0.753,329933.0,0.934,0.06500,0.1280,-3.011,0.0736,113.525,0.730
3,3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),2001-03-27,80,0.15300,0.481,259933.0,0.638,0.00000,0.1540,-5.862,0.0276,79.064,0.497
4,4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,2001-10-02,49,0.40100,0.700,252933.0,0.631,0.00000,0.1190,-6.085,0.1070,83.403,0.558


In [23]:
billboard_df = billboard_df.drop(columns=["Unnamed: 0"])

In [24]:
spotify_df['hit'] = 0
billboard_df['hit'] = 1

In [25]:
df = pd.concat([billboard_df, spotify_df], ignore_index=True)
df

,id,artist_name,key,mode,track_name,rel_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,hit
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,2000,68,0.00118,0.541,216067.0,0.864,0.00000,0.0896,-4.915,0.0357,124.557,0.435,1
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',2001-06-05,70,0.26300,0.652,210200.0,0.609,0.00101,0.2330,-7.519,0.0370,95.986,0.482,1
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,2001-01-01,67,0.01740,0.753,329933.0,0.934,0.06500,0.1280,-3.011,0.0736,113.525,0.730,1
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),2001-03-27,80,0.15300,0.481,259933.0,0.638,0.00000,0.1540,-5.862,0.0276,79.064,0.497,1
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,2001-10-02,49,0.40100,0.700,252933.0,0.631,0.00000,0.1190,-6.085,0.1070,83.403,0.558,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20226,46LhBf6TvYjZU2SMvGZAbn,DJ Combo,6.0,0.0,The One,2020-12-25,0,0.00917,0.792,147615.0,0.866,0.00006,0.1780,-5.089,0.0356,125.972,0.186,0
20227,7tue2Wemjd0FZzRtDrQFZd,Alessia Cara,4.0,1.0,A Little More,2021-01-22,0,0.79500,0.429,144720.0,0.211,0.00000,0.1960,-11.665,0.0360,94.710,0.228,0
20228,48Qj61hOdYmUCFJbpQ29Ob,Roger Fly,4.0,0.0,Together,2020-12-09,0,0.80600,0.671,218147.0,0.589,0.92000,0.1130,-12.393,0.0282,108.058,0.714,0
20229,1gcyHQpBQ1lfXGdhZmWrHP,Taylor Swift,0.0,1.0,champagne problems,2021-01-07,69,0.92000,0.462,244000.0,0.240,0.00000,0.1130,-12.077,0.0377,171.319,0.320,0


In [26]:
df = df.drop_duplicates('id', keep='first')
df.shape

(18262, 18)

In [27]:
df["hit"].value_counts()

0    16360
1     1902
Name: hit, dtype: int64

In [28]:
def extract_year(date):
    year = date.split('-')[0]
    return int(year)

df["year"] = df["rel_date"].apply(extract_year)
df.head()

/home/triloq/anaconda2/envs/mldl/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,artist_name,key,mode,track_name,rel_date,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,hit,year
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,2000,68,0.00118,0.541,216067.0,0.864,0.00000,0.0896,-4.915,0.0357,124.557,0.435,1,2000
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',2001-06-05,70,0.26300,0.652,210200.0,0.609,0.00101,0.2330,-7.519,0.0370,95.986,0.482,1,2001
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,2001-01-01,67,0.01740,0.753,329933.0,0.934,0.06500,0.1280,-3.011,0.0736,113.525,0.730,1,2001
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),2001-03-27,80,0.15300,0.481,259933.0,0.638,0.00000,0.1540,-5.862,0.0276,79.064,0.497,1,2001
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,2001-10-02,49,0.40100,0.700,252933.0,0.631,0.00000,0.1190,-6.085,0.1070,83.403,0.558,1,2001


In [29]:
df = df.drop(columns=["rel_date"])

In [30]:
api_key = '7d8043070d8b187e54fbcb656e8537aa'
musixmatch = Musixmatch(api_key)


df['lyrics'] = np.nan
df['explicit'] = np.nan

lyrics_list = []
explicit_list = []
errors = []

In [31]:
songs = list(df.track_name)
artists = list(df.artist_name)

def get_song_lyrics(track, artist):
    # Get lyrics
    resp = musixmatch.matcher_lyrics_get(track, artist)
    
    # If empty response
    if (resp['message']['body'] == '') or (resp['message']['body'] == []):
        return None
    else:
        lyrics = resp['message']['body']['lyrics']['lyrics_body'].split('***')[0]
        explicit = resp['message']['body']['lyrics']['explicit']
        return [lyrics, explicit]

In [32]:
lyrics_cache = {}
lyrics_cache_file = "cache/lyrics.cache"
try:
    if os.path.exists(lyrics_cache_file):
        with open(lyrics_cache_file, 'rb') as cache:
            lyrics_cache = pickle.load(cache)
        
    for i in tqdm_notebook(range(len(songs))):
        song = songs[i]
        artist = artists[i]

        key = song+artist
        if key in lyrics_cache:
            resp = lyrics_cache[key]
        else:
            resp = get_song_lyrics(song, artist)
            lyrics_cache[key] = resp
        if resp == None:
            lyrics_list.append(np.NaN)
            explicit_list.append(np.NaN)
            errors.append(i)
        else:
            lyrics_list.append(resp[0])
            explicit_list.append(resp[1])

except Exception as e:
    print(e)            

finally:
    with open(lyrics_cache_file, 'wb') as cache:
        pickle.dump(lyrics_cache, cache)
        print('Cache saved with size', len(lyrics_cache))
        lyrics_cache.clear()

/home/triloq/anaconda2/envs/mldl/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/18262 [00:00<?, ?it/s]

Cache saved with size 17190


In [33]:
df.lyrics = lyrics_list
df.explicit = explicit_list

In [34]:
df

,id,artist_name,key,mode,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,hit,year,lyrics,explicit
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,68,0.00118,0.541,216067.0,0.864,0.00000,0.0896,-4.915,0.0357,124.557,0.435,1,2000,Desperate for changing\nStarving for truth\nI'...,0.0
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',70,0.26300,0.652,210200.0,0.609,0.00101,0.2330,-7.519,0.0370,95.986,0.482,1,2001,I keep on fallin' in\nAnd out of love with you...,0.0
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,67,0.01740,0.753,329933.0,0.934,0.06500,0.1280,-3.011,0.0736,113.525,0.730,1,2001,"All my girls at the party, look at that body\n...",0.0
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),80,0.15300,0.481,259933.0,0.638,0.00000,0.1540,-5.862,0.0276,79.064,0.497,1,2001,Now that she's back in the atmosphere\nWith dr...,0.0
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,49,0.40100,0.700,252933.0,0.631,0.00000,0.1190,-6.085,0.1070,83.403,0.558,1,2001,Murder Inc.\nWhat's my motherfuckin'name\nR-U-...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221,45XnLMuqf3vRfskEAMUeCH,Alessia Cara,4.0,1.0,A Little More,0,0.79500,0.429,144720.0,0.211,0.00000,0.1960,-11.665,0.0360,94.710,0.228,0,2021,There you are with your college friends\n\nYou...,0.0
20223,4pPFI9jsguIh3wC7Otoyy8,Alessia Cara,4.0,1.0,A Little More,0,0.79500,0.429,144720.0,0.211,0.00000,0.1960,-11.665,0.0360,94.710,0.228,0,2021,There you are with your college friends\n\nYou...,0.0
20225,52YtxLVUyvtiGPxwwxayHZ,Alessia Cara,4.0,1.0,A Little More,0,0.79500,0.429,144720.0,0.211,0.00000,0.1960,-11.665,0.0360,94.710,0.228,0,2021,There you are with your college friends\n\nYou...,0.0
20227,7tue2Wemjd0FZzRtDrQFZd,Alessia Cara,4.0,1.0,A Little More,0,0.79500,0.429,144720.0,0.211,0.00000,0.1960,-11.665,0.0360,94.710,0.228,0,2021,There you are with your college friends\n\nYou...,0.0


In [35]:
ddf = df.dropna()
ddf['hit'].value_counts(sort=True)

0    9364
1    1803
Name: hit, dtype: int64

In [36]:
ddf.shape

(11167, 20)

In [37]:
ddf = ddf.drop_duplicates(subset = ['artist_name', 'track_name', 'lyrics'], keep = 'first').reset_index(drop = True)
ddf.shape

(10280, 20)

In [38]:
ddf.head(10)

,id,artist_name,key,mode,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,hit,year,lyrics,explicit
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,68,0.00118,0.541,216067.0,0.864,0.000000,0.0896,-4.915,0.0357,124.557,0.435,1,2000,Desperate for changing\nStarving for truth\nI'...,0.0
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',70,0.26300,0.652,210200.0,0.609,0.001010,0.2330,-7.519,0.0370,95.986,0.482,1,2001,I keep on fallin' in\nAnd out of love with you...,0.0
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,67,0.01740,0.753,329933.0,0.934,0.065000,0.1280,-3.011,0.0736,113.525,0.730,1,2001,"All my girls at the party, look at that body\n...",0.0
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),80,0.15300,0.481,259933.0,0.638,0.000000,0.1540,-5.862,0.0276,79.064,0.497,1,2001,Now that she's back in the atmosphere\nWith dr...,0.0
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,49,0.40100,0.700,252933.0,0.631,0.000000,0.1190,-6.085,0.1070,83.403,0.558,1,2001,Murder Inc.\nWhat's my motherfuckin'name\nR-U-...,1.0
5,2f5N826udWfjT9iomeaBJt,Matchbox Twenty,9.0,1.0,If You're Gone,66,0.42700,0.544,274933.0,0.659,0.000001,0.1260,-7.191,0.0298,109.933,0.361,1,2000,I think I've already lost you\nI think you're ...,1.0
6,3RmKpob8xzv1pzHEQrMJah,Eve,8.0,0.0,Let Me Blow Ya Mind,75,0.24200,0.908,230133.0,0.557,0.000000,0.0709,-4.243,0.1070,90.032,0.897,1,2001,"Uh, uh, uh, huh\nYo, yo\n\nDrop your glasses, ...",1.0
7,751gBcu62kORDelX7FV0mM,Dido,1.0,0.0,Thank You,62,0.28400,0.722,217827.0,0.595,0.000540,0.0694,-9.928,0.0415,79.981,0.738,1,1999,"My tea's gone cold, I'm wonderin' why I\nGot o...",1.0
8,5c1QJAiC3773EwiZLwHf5x,Lenny Kravitz,2.0,1.0,Again,71,0.01480,0.550,231667.0,0.804,0.000043,0.1050,-5.218,0.0271,79.166,0.789,1,2000,I've been searching for you\nI heard a cry wit...,0.0
9,4zYTZvtcww7OWKie7fxA9E,Blu Cantrell,5.0,0.0,Hit 'Em Up Style (Oops!),67,0.20100,0.667,250707.0,0.773,0.000000,0.4040,-4.983,0.0586,89.976,0.667,1,2001,While he was scheming\nI was beamin' in the be...,1.0


In [39]:
ddf = ddf[ddf["lyrics"] != ''].reset_index(drop=True)

In [40]:
ddf

,id,artist_name,key,mode,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,hit,year,lyrics,explicit
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,68,0.001180,0.541,216067.0,0.864,0.00000,0.0896,-4.915,0.0357,124.557,0.435,1,2000,Desperate for changing\nStarving for truth\nI'...,0.0
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',70,0.263000,0.652,210200.0,0.609,0.00101,0.2330,-7.519,0.0370,95.986,0.482,1,2001,I keep on fallin' in\nAnd out of love with you...,0.0
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,67,0.017400,0.753,329933.0,0.934,0.06500,0.1280,-3.011,0.0736,113.525,0.730,1,2001,"All my girls at the party, look at that body\n...",0.0
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),80,0.153000,0.481,259933.0,0.638,0.00000,0.1540,-5.862,0.0276,79.064,0.497,1,2001,Now that she's back in the atmosphere\nWith dr...,0.0
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,49,0.401000,0.700,252933.0,0.631,0.00000,0.1190,-6.085,0.1070,83.403,0.558,1,2001,Murder Inc.\nWhat's my motherfuckin'name\nR-U-...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9264,3NUmUIyzNLBp8bCFMH8Mif,Foo Fighters,7.0,1.0,Waiting On A War,69,0.009840,0.530,253840.0,0.759,0.00000,0.3190,-7.067,0.0351,131.999,0.502,0,2021,I've been waiting on a war since I was young\n...,0.0
9265,0fJ1caLzidzTlIL3pPX1eU,Jazmine Sullivan,2.0,1.0,Precious' Tale,59,0.715000,0.734,43320.0,0.346,0.00000,0.3940,-11.722,0.3550,88.849,0.930,0,2021,To be honest\nMoney makes me cum\nIt's like I ...,1.0
9266,0VCOLfLPC9oOrc90WGp2r3,Sean Paul,7.0,1.0,So Fine - TC Remix,0,0.000032,0.530,292427.0,0.959,0.02730,0.3550,-5.648,0.1440,173.009,0.262,0,2021,"Girl, girl, I'll be your sunshine, you can be ...",1.0
9267,660rulYF3eLCuW6rQpiMdL,Ashnikko,1.0,0.0,Little Boy,61,0.105000,0.781,172720.0,0.487,0.00000,0.0802,-7.301,0.1670,129.941,0.327,0,2021,"Oh, you think you're worth all the fuss?\nYou ...",1.0


In [41]:
ddf['hit'].value_counts(sort=True)

0    7592
1    1677
Name: hit, dtype: int64

In [42]:
# Convert text to lower case
ddf["lyrics"] = ddf["lyrics"].str.lower()

# Remove numbers
ddf["lyrics"] = ddf["lyrics"].str.replace('\d+', '')

# Remove stop words
ddf["lyrics"] = ddf["lyrics"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords.words('english')))

# Strip whitespaces
ddf["lyrics"] = ddf["lyrics"].str.strip()
# Remove punctuations
# ddf["lyrics"] = ddf["lyrics"].str.replace('[^\w\s]','')

# lemmatizer = WordNetLemmatizer()
# def lemmatize_sentences(sentence):
#     tokens = sentence.split()
#     # stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     return ' '.join(lemmatized_tokens)

# ddf['lyrics'] = ddf['lyrics'].apply(lemmatize_sentences)

# ddf["lyrics1"].apply(lambda x: [stemmer.stem(y) for y in x])

# ddf["lyrics"]

In [43]:
analyzer = SentimentIntensityAnalyzer()

ddf['sentimentPos'] = ddf['lyrics'].apply(lambda x: analyzer.polarity_scores(x)['pos'])
ddf['sentimentNeg'] = ddf['lyrics'].apply(lambda x: analyzer.polarity_scores(x)['neg'])
ddf['sentimentComp'] = ddf['lyrics'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
ddf['sentimentNeu'] = ddf['lyrics'].apply(lambda x: analyzer.polarity_scores(x)['neu'])

# We can also create a feature that is the difference between the positive and the negative score
ddf['pos_neg_diff'] = ddf['sentimentPos'] - ddf['sentimentNeg']
ddf

,id,artist_name,key,mode,track_name,popularity,acousticness,danceability,duration_ms,energy,...,valence,hit,year,lyrics,explicit,sentimentPos,sentimentNeg,sentimentComp,sentimentNeu,pos_neg_diff
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,68,0.001180,0.541,216067.0,0.864,...,0.435,1,2000,desperate changing starving truth i'm closer s...,0.0,0.198,0.154,0.6597,0.648,0.044
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',70,0.263000,0.652,210200.0,0.609,...,0.482,1,2001,keep fallin' love sometimes love sometimes mak...,0.0,0.418,0.116,0.9456,0.467,0.302
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,67,0.017400,0.753,329933.0,0.934,...,0.730,1,2001,"girls party, look body shakin' thing like neve...",0.0,0.229,0.133,0.8520,0.638,0.096
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),80,0.153000,0.481,259933.0,0.638,...,0.497,1,2001,back atmosphere drops jupiter hair he-e-ey act...,0.0,0.210,0.000,0.9288,0.790,0.210
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,49,0.401000,0.700,252933.0,0.631,...,0.558,1,2001,murder inc. what's motherfuckin'name r-u-l-e b...,1.0,0.232,0.194,0.5832,0.574,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9264,3NUmUIyzNLBp8bCFMH8Mif,Foo Fighters,7.0,1.0,Waiting On A War,69,0.009840,0.530,253840.0,0.759,...,0.502,0,2021,i've waiting war since young since little boy ...,0.0,0.129,0.290,-0.8599,0.582,-0.161
9265,0fJ1caLzidzTlIL3pPX1eU,Jazmine Sullivan,2.0,1.0,Precious' Tale,59,0.715000,0.734,43320.0,0.346,...,0.930,0,2021,honest money makes cum like feel like see man ...,1.0,0.316,0.000,0.8074,0.684,0.316
9266,0VCOLfLPC9oOrc90WGp2r3,Sean Paul,7.0,1.0,So Fine - TC Remix,0,0.000032,0.530,292427.0,0.959,...,0.262,0,2021,"girl, girl, i'll sunshine, mine wind chime, i'...",1.0,0.282,0.083,0.9761,0.636,0.199
9267,660rulYF3eLCuW6rQpiMdL,Ashnikko,1.0,0.0,Little Boy,61,0.105000,0.781,172720.0,0.487,...,0.327,0,2021,"oh, think worth fuss? think bitch settled like...",1.0,0.213,0.197,0.1381,0.590,0.016


In [37]:
# count_vect = CountVectorizer()
# count_matrix = count_vect.fit_transform(ddf["lyrics"])

# count_array = count_matrix.toarray()

# ddf["lyrics1"] = count_array.tolist()
# Dropping original lryics column
# ddf.drop(columns=["lyrics"], inplace=True)
# Rename vectorized lyrics column to `lyrics`
# ddf.rename(columns={"lyrics1": "lyrics"}, inplace=True)


In [44]:
# Dropping original lryics column
ddf.drop(columns=["lyrics"], inplace=True)

In [46]:
modelCreationData = ddf[ddf.year < 2020]
testData = ddf[ddf.year >= 2020]
print(modelCreationData.shape)
print(testData.shape)

(7838, 24)
(1431, 24)


In [48]:
modelCreationData.to_csv("data/modelCreationData.csv", index=False)
testData.to_csv("data/test.csv", index=False)